In [1]:
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled
    
    
# Load various imports 
import pandas as pd
import os
import librosa
import numpy as np
# Set the path to the full UrbanSound dataset 
fulldatasetpath = './FolderFrames/MrsPiggyAudioTrainingSet'

features = []
pathNoMrsPiggy = './FolderFrames/MrsPiggyAudioTrainingSet/NoMrsPiggy/'
pathYesMrsPiggy = './FolderFrames/MrsPiggyAudioTrainingSet/YesMrsPiggy/'
    
# Iterate through each sound file and extract the features
for filename in os.listdir(pathNoMrsPiggy):
    if filename.endswith(".wav"):
        path = pathNoMrsPiggy + filename
        data = extract_features(path)
        features.append([data, "NoMrsPiggy"])

for filename in os.listdir(pathYesMrsPiggy):
    if filename.endswith(".wav"):
        path = pathYesMrsPiggy + filename
        data = extract_features(path)
        features.append([data, "YesMrsPiggy"])
        

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

Finished feature extraction from  1622  files


In [25]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import numpy as np

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1297, 40)
(325, 40)
(1297, 2)
(325, 2)


In [51]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 1
num_channels = 1
print("ts", x_train.shape[0])

x_train = x_train.reshape(x_train.shape[0], num_rows, num_channels, 1)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_channels, 1)

print("tss", x_train.shape)


num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

ts 1297
tss (1297, 40, 1, 1)


ValueError: Input 0 is incompatible with layer conv2d_38: expected ndim=4, found ndim=3

In [39]:
# Compile the model
x_train = np.expand_dims(x_train, -1)
print("xtrain:", x_train.shape)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.fit(x=x_train, y=y_train, batch_size=None, epochs=1, verbose=1, validation_data=(x_test, y_test))
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
#score = model.evaluate(x_test, y_test, verbose=1)
#accuracy = 100*score[1]
#
#print("Pre-training accuracy: %.4f%%" % accuracy) 

xtrain: (1297, 40, 1, 1, 1)


AttributeError: 'Sequential' object has no attribute '_feed_input_names'